In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

import numpy as np
import torch
import librosa
from torch.nn import functional as F
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

from wavenet import model, train, sample, audio, datasets, utils, viz

# Learning Rate Finder

Use this to find a good learning rate for the given dataset and hyperparameters. This will log to wandb, have a look at the graph there. It will not be plotted on a log scale or smoothed. Good enough though. 

In [ ]:
dry = False
if dry:
    os.environ['WANDB_MODE'] = 'dryrun'

In [ ]:
p = model.HParams(n_audio_chans=2)
X, X_test = datasets.Sines(640, 1, p, hz=440, amp=1.0), datasets.Sines(640, 1, p, hz=440, amp=1.0)
X_batch = torch.stack([X[i] for i in range(len(X))])

In [ ]:
track_i = viz.plot_track(X_batch, n_samples=p.receptive_field_size())

In [ ]:
ipd.Audio(X_batch[track_i], rate=p.sampling_rate)

In [ ]:
m = model.Wavenet(p)
t = train.Trainer(m, X, X_test, train.HParams(max_epochs=3, batch_size=8, num_workers=8, finder=True), None)

In [ ]:
t.train()